In [1]:
import pandas as pd
import datetime

In [2]:
users_df = pd.read_csv('takehome_users.csv', encoding='latin-1')
user_engagement = pd.read_csv('takehome_user_engagement.csv')

In [3]:
users_df

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


In [4]:
user_engagement

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [5]:
ue = user_engagement.copy()
ue['time_stamp'] = pd.to_datetime(ue['time_stamp'])
ue

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [6]:
df = ue.sort_values(by=['user_id', 'time_stamp'])

df['time_diff'] = df.groupby('user_id')['time_stamp'].diff()

mask = df['time_diff'] <= pd.Timedelta(days=7)

df['visit_count_7days'] = df.groupby('user_id')['visited'].cumsum()

filtered_df = df[df['visit_count_7days'] >= 3].copy()

filtered_df.drop(['time_diff', 'visit_count_7days'], axis=1, inplace=True)

filtered_df.reset_index(drop=True, inplace=True)

filtered_df

,time_stamp,user_id,visited
0,2013-12-09 03:45:04,2,1
1,2013-12-25 03:45:04,2,1
2,2013-12-31 03:45:04,2,1
3,2014-01-08 03:45:04,2,1
4,2014-02-03 03:45:04,2,1
...,...,...,...
196501,2014-05-27 11:04:47,11988,1
196502,2014-06-01 11:04:47,11988,1
196503,2014-04-28 14:22:45,11991,1
196504,2014-05-17 14:22:45,11991,1


In [7]:
len(filtered_df['user_id'].unique())

2248

In [8]:
# Find which factors are most useful in predicting who will become an adopted user

In [9]:
merged = users_df.merge(filtered_df, how='inner', left_on='object_id', right_on='user_id')
merged

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_stamp,user_id,visited
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-12-09 03:45:04,2,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-12-25 03:45:04,2,1
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2013-12-31 03:45:04,2,1
3,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-01-08 03:45:04,2,1
4,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-02-03 03:45:04,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196501,11988,2014-03-15 11:04:47,Minick John,JohnFMinick@yahoo.com,PERSONAL_PROJECTS,1.401621e+09,0,0,114,NaN,2014-05-27 11:04:47,11988,1
196502,11988,2014-03-15 11:04:47,Minick John,JohnFMinick@yahoo.com,PERSONAL_PROJECTS,1.401621e+09,0,0,114,NaN,2014-06-01 11:04:47,11988,1
196503,11991,2014-04-20 14:22:45,Vaughn Arthur,ArthurJVaughn@gmail.com,SIGNUP_GOOGLE_AUTH,1.401460e+09,0,0,388,NaN,2014-04-28 14:22:45,11991,1
196504,11991,2014-04-20 14:22:45,Vaughn Arthur,ArthurJVaughn@gmail.com,SIGNUP_GOOGLE_AUTH,1.401460e+09,0,0,388,NaN,2014-05-17 14:22:45,11991,1


In [10]:
merged['creation_source'].value_counts()

ORG_INVITE            65997
GUEST_INVITE          44631
SIGNUP                35168
SIGNUP_GOOGLE_AUTH    32515
PERSONAL_PROJECTS     18195
Name: creation_source, dtype: int64

In [11]:
merged['opted_in_to_mailing_list'].value_counts()

0    145725
1     50781
Name: opted_in_to_mailing_list, dtype: int64

In [12]:
merged['enabled_for_marketing_drip'].value_counts()

0    166117
1     30389
Name: enabled_for_marketing_drip, dtype: int64

In [13]:
merged['org_id'].value_counts()

1      2099
13     2032
62     2026
4      1918
7      1809
       ... 
299       2
329       2
348       2
310       1
365       1
Name: org_id, Length: 412, dtype: int64

In [14]:
merged['invited_by_user_id'].value_counts()

4019.0    912
7816.0    898
3623.0    736
7358.0    705
6312.0    696
         ... 
3477.0      1
9180.0      1
6701.0      1
3384.0      1
9550.0      1
Name: invited_by_user_id, Length: 995, dtype: int64